![alt text](./pageheader_rose2_babies.jpg)

# Data Science in Medicine using Python

### Author: Dr Gusztav Belteki

## 1. Review of homework: slicing and dicing in Python

Subsetting and indexing pandas DataFrames

In [ ]:
import os
import pandas as pd

path = os.path.join('data', 'CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip',)
data = pd.read_csv(path)
data

In [ ]:
# Select the third row only

selection = data
selection.iloc[2]

In [ ]:
# Select the "MVe [L/min]" column only"

selection = data['5001|MVe [L/min]']
selection

In [ ]:
data.columns

In [ ]:
# Select the "MVe [L/min]" and "MVi [L/min]" columns only

columns_to_keep = ['5001|MVe [L/min]', '5001|MVi [L/min]']
selection = data[columns_to_keep]
selection

In [ ]:
# Select the 'MVe [L/min]' value from the third row

selection = data.iloc[2]['5001|MVe [L/min]']
selection

##  End-to-end analysis of tabular (two-dimensional data) using pandas

In [ ]:
pd.read_csv? # You can also look it up on the Internet

In [ ]:
import os
import pandas as pd

path = os.path.join('data', 'CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip',)
data = pd.read_csv(path)
data

In [ ]:
len(data)

In [ ]:
data.shape

In [ ]:
data.ndim

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.head(10)

In [ ]:
data.tail(10)

### What is the problem with these data

- Indexed by numbers only (uninformative, it should be indexed by date and time)
- Date and time are in separate columnns
- Date and time formats are not appropriate
- Column names are too long and difficult to read
- Lots of `na` values 
- half of every row is empty
- Some columns have barely any informative values
- Some values are not meaningful (e.g. tidal volume should be mL/kg not mL)

*We will deal with all these issues*

In [ ]:
data.info()

### 1. Convert the `date` and `time` columns to appropriate format

In [ ]:
%%time # Let us time how long the import takes

path = os.path.join('data', 'CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip')
data = pd.read_csv(path)
data

This takes much longer

In [ ]:
%%time 

path = os.path.join('data', 'CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip')
data = pd.read_csv(path, parse_dates = ['Date', 'Time'])
data

In [ ]:
data.info()

There must be a better way !!!

Google: **"How to combine date and time columns in pandas"**

In [ ]:
%%time

path = os.path.join('data', 'CsvLogBase_2020-11-02_134238.904_slow_Measurement.csv.zip')
data = pd.read_csv(path, parse_dates = [['Date', 'Time']])
data

In [ ]:
data.info()

### 2. Set the `Date_Time` column as index

In [ ]:
data.index

In [ ]:
list(data.index)[:10]

In [ ]:
data = data.set_index('Date_Time') # Do not use inplace modifications
data

In [ ]:
data.index

### 3. Change the clumsy column names

In [ ]:
data.columns

We could just replace it with 

`data.columns = ['...', '...', '...']` 

but that is error prone

In [ ]:
# Welcome to list comprehensions

new_columns_1 = [item for item in data.columns]
print(new_columns_1)

In [ ]:
new_columns_2 = [item[5:] for item in data.columns]
print(new_columns_2)

In [ ]:
new_columns_3 = [item[5:] for item in data.columns if item.startswith('5001')]
print(new_columns_3)

In [ ]:
# The expression to the right of `=` is evaluated firs (before assignment)

new_columns_3 = ['Time [ms]', 'Rel.Time [s]'] + new_columns_3
new_columns_3

In [ ]:
data.columns = new_columns_3
data.columns

In [ ]:
data.head()

### 4. Combine consecutive columns as they contain complementary data

In [ ]:
data.info()

In [ ]:
data.head(10)

In [ ]:
# This is called a `hack`
# During `mean`() na values are excluded by default

data = data.resample('1S').mean()
data.head(10)

In [ ]:
data.info()

##### Please recognise that these are already aggregate data !!! The numbers in the same colum do not necessary belong to the same observation (e.g. ventilator inflations)

### 5. Remove na values

In [ ]:
data.info()

In [ ]:
# Some columns are almost completely empty and hopeless - drop them

data = data.drop(['Tispon [s]', 'I:Espon (I-Part) [no unit]', 
                  'I:Espon (E-Part) [no unit]'], axis = 1)

In [ ]:
data.info()

In [ ]:
# How many data points are missing ?

data.isnull().sum()

In [ ]:
# How many percent of data is missing

data.isnull().sum() / len(data) * 100

#### A lot of things are happening here, for example vectorized computation, broadcasting We will speak about them during the next session.

### 6. Now let us save the processed data

#### A. export them as `csv` files

In [ ]:
%%time

data.to_csv('data_processed')

In [ ]:
%%time

path = os.path.join('data', 'data_processed')
data.to_csv(path)

#### B. export az Excel file

This will run for a long time

In [ ]:
%%time

path = os.path.join('data', 'data_processed.xlsx')
data.to_excel(path, sheet_name = 'processed_data')

#### C. export them as serialised binary data  - `pickle`

In [ ]:
%%time

import pickle

path = os.path.join('data', 'data_processed.pickle')
filehandle = open(os.path.join(path), 'wb')
pickle.dump(data, filehandle)
filehandle.close()

In [ ]:
%%time

import pickle

path = os.path.join('data', 'data_processed.pickle')
filehandle = open(os.path.join(path), 'rb')
data_processed = pickle.load(filehandle)
filehandle.close()

data_processed.head()

##### To be continued... 

### 6. Homework

### 1. Further explorative analysis on data

In [ ]:
# Select all data during the 1 minute period at 2020-11-03 13:00 

selection = data
selection

In [ ]:
# Select all data between 2020-11-03 13:00 and and 15:00

selection = data
selection

In [ ]:
# Select all data between 2020-11-03 13:00 and and 15:00 and limit it to 
# "MVe [L/min]" and "MVi [L/min]" columns only

selection = data
selection

### 2. Import and process the `Radiometer_TCM_sample_data.xlsx'` file containing transcuatenous CO2 data into a Dataframe named "data_2"

The file is located in the "data" subfolder

*Hint: the first 8 rows contain text rather than data and need to be skipped*

Check the data types of the various columns

Now the 'Time` column seems to be in the correct format. Set it as index

 ...but most other columns are just objects. First check what values do the columns contain

... to be continued